# Yemen Accessibility to Health
## UTM zone 38

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys, os, inspect, logging, importlib

import geopandas as gpd
import pandas as pd
import numpy as np
import osmnx as ox
import networkx as nx
from shapely.ops import split, unary_union
from shapely.geometry import box, Point

import matplotlib.pyplot as plt

In [3]:
# Get reference to GOSTNets
sys.path.append(r'../../../../GOSTnets')
import GOSTnets as gn

In [4]:
from GOSTnets.load_osm import *

In [5]:
measure_crs = 'EPSG:2089'

## Import clipped Graphs without speeds

In [6]:
G2 = nx.read_gpickle(r'yemen_regular_graph_wo_speeds3/yemen_regular_graph_wo_speeds3.pickle')

In [7]:
print(nx.info(G2))

MultiDiGraph with 142013 nodes and 355193 edges


In [8]:
gn.example_edge(G2)

('1', '110147', {'geometry': <shapely.geometry.linestring.LineString object at 0x7fa956089940>, 'id': 201859, 'one_way': False, 'osm_id': '810908760', 'Type': 'legitimate', 'infra_type': 'unclassified', 'key': 'edge_201859', 'length': 0.061233766945625, 'name': None})


## Import Origins

In [9]:
origins = gpd.read_file(r'input/hotosm_yem_populated_places_points.shp')

### split the origins between the 2 UTM zones in Yemen

In [10]:
# UTM zone 38
origins = origins.cx[42:48, 0:90]
# UTM zone 39
#origins = origins.cx[48:54, 0:90]

## Adv Snap origins to graph

In [11]:
%time
G_adv, pois_meter, new_footway_edges = gn.advanced_snap(G2, origins, u_tag = 'stnode', v_tag = 'endnode', node_key_col='osmid', edge_key_col='osm_id', poi_key_col='osm_id', road_col = 'infra_type', oneway_tag = 'one_way', path='output/yemen_38n', threshold=10000, knn=5, measure_crs=measure_crs, factor = 1000, verbose = True)

CPU times: user 1 µs, sys: 1 µs, total: 2 µs
Wall time: 3.1 µs
POI keys are not ints, are you sure this is okay?
Building rtree...
finished Building rtree
seconds elapsed: 23.09592866897583
updating external nodes (pois)
finished updating external nodes (pois)
seconds elapsed: 23.11693501472473
Projecting POIs to the network...2
finished pois_meter['near_idx'] and pois_meter['near_lines']
seconds elapsed: 51.98463821411133
finished pois_meter['kne_idx']
seconds elapsed: 54.72537589073181
assigning a projected point to each POI


/opt/conda/envs/geo_env/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:118: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


finished assigning a projected point to each POI
seconds elapsed: 58.418288230895996
Updating internal nodes...
finished Updating internal nodes
seconds elapsed: 62.97144913673401
print _new_nodes
Updating internal edges...
finished creating line_pps_dict
seconds elapsed: 65.30347681045532
creating new_lines


/home/Implementations/FY22/ACC_yemen_accessibility/../../../../GOSTnets/GOSTnets/core.py:2983: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  new_lines = list(split(line, pps))  # split into segments
/home/Implementations/FY22/ACC_yemen_accessibility/../../../../GOSTnets/GOSTnets/core.py:2983: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  new_lines = list(split(line, pps))  # split into segments


finished creating new_lines
seconds elapsed: 73.77832007408142
Updating update_edges


/opt/conda/envs/geo_env/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:118: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


debugging
finished Updating update_edges
seconds elapsed: 82.39592027664185
Updating external links...


/opt/conda/envs/geo_env/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:118: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


debugging
node count before: 271311
node count after: 271311
pois_meter count before: 79334
pois_meter count after: 79334
Remove edge projections greater than threshold: 22/79334 (0.03%)
finished Updating external links
seconds elapsed: 92.71542739868164
NOTE: duplication in node coordinates keys
Nodes count: 271311
Node coordinates key count: 271193
Missing 'from' nodes: 0
Missing 'to' nodes: 0
making a new graph from edges and nodes


In [12]:
originNodes = list(pois_meter['osm_id'])

In [13]:
pois_meter[:3]

,osm_id,population,place,name,source,is_in,geometry,near_idx,near_lines,kne_idx,pp
1,258013552.0,1527861,city,صنعاء,None,None,POINT (44.20588 15.35386),"[146631, 181104, 160498, 160499, 143940]","146631 LINESTRING (414747.940 1697676.385, ...",181104,POINT (414810.6064762817 1697613.3285479401)
2,263261532.0,458933,city,تعز,None,None,POINT (44.02153 13.57522),"[76936, 191683, 11377, 76937, 110716]","76936 LINESTRING (394254.203 1500985.653, ...",76936,POINT (394136.83818937716 1500979.8546253112)
3,263263790.0,508489,city,عدن,None,None,POINT (45.02850 12.78959),"[69025, 192744, 174749, 174748, 166590]","69025 LINESTRING (503267.829 1413535.563, ...",69025,POINT (503055.60754716926 1413688.2912471865)


In [14]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)

### Export to pickle to be used when calculating OD Matrix using graph-tool
CSVs don’t have types, that information is never preserved. CSVs only have strings. This is why we are saving as a pickle. Some of the destinations are strings, and some are ints.

In [15]:
pois_meter_export = pois_meter[['geometry','osm_id']]
pois_meter_export
#pois_meter_export[:10].to_pickle('input2/origins.pickle')
pois_meter_export.to_pickle('input2/origins_38N.pickle')

### Add walking time to new footway edges

In [16]:
yemen_travel_times = {
'motorway': 40,
'motorway_link': 40,
'trunk': 40,
'trunk_link':40,
'primary': 37.5,
'primary_link': 37.5,
'secondary': 35,
'secondary_link': 35,
'tertiary':30,
'tertiary_link':30,
'residential': 20,
'unclassified': 20,
'projected_footway':3.5
}

In [17]:
# convert_network_to_time function now has 'projected_footway' as a highway class
G_adv_w_time = gn.convert_network_to_time(G_adv, 'length', road_col = 'infra_type', speed_dict = yemen_travel_times, factor=1000)

In [18]:
G_adv_w_time.nodes['20768']

{'x': 49.1780323,
 'y': 14.611084599999996,
 'geometry': <shapely.geometry.point.Point at 0x7fa918f14700>,
 'infra_type': nan}

### Advanced snapping to rehab graph

In [19]:
G_adv_w_time_rehab = G_adv_w_time.copy()

### Alter times on rehab roads for rehab graph

In [20]:
count=0
for u, v, data in G_adv_w_time_rehab.edges(data=True):
    if 'Type' in data:
        if data['Type'] == 'rehab':
            print(data['Type'])
            count+=1
            # data['length'] is in meters
            # speed after rehab is 80 km per hr
            kmph = (data['length'] / 1000) / 80
            in_seconds = kmph * 60 * 60
            print(f"time before is {data['time']}")
            data['time'] = in_seconds
            print(f"time after is {data['time']}")

rehab
time before is 69.6354948802647
time after is 17.408873720066175
rehab
time before is 69.6354948802647
time after is 17.408873720066175
rehab
time before is 31.133936273084636
time after is 7.783484068271159
rehab
time before is 88.00626675737529
time after is 44.00313337868764
rehab
time before is 92.8588671647334
time after is 46.4294335823667
rehab
time before is 42.402747833508684
time after is 21.201373916754342
rehab
time before is 88.00626675737529
time after is 44.00313337868764
rehab
time before is 92.8588671647334
time after is 46.4294335823667
rehab
time before is 22.39558924684761
time after is 11.197794623423805
rehab
time before is 43.082475780236486
time after is 21.541237890118243
rehab
time before is 42.402747833508684
time after is 21.201373916754342
rehab
time before is 35.36145011215584
time after is 8.84036252803896
rehab
time before is 40.84681876571943
time after is 10.211704691429858
rehab
time before is 98.94135870188461
time after is 24.735339675471153
r

rehab
time before is 15.791881210808066
time after is 7.402444317566279
rehab
time before is 9.80235112239696
time after is 4.594852088623575
rehab
time before is 3.364255038793728
time after is 1.5769945494345599
rehab
time before is 8.09210269576233
time after is 4.046051347881165
rehab
time before is 2.7442765605492
time after is 1.3721382802746
rehab
time before is 8.157020083031828
time after is 3.5686962863264253
rehab
time before is 10.5620797384092
time after is 4.620909885554025
rehab
time before is 5.0314567722159085
time after is 2.2012623378444602
rehab
time before is 4.489941792833795
time after is 1.964349534364785
rehab
time before is 19.252853722273784
time after is 8.423123503494779
rehab
time before is 130.37586903085713
time after is 65.18793451542857
rehab
time before is 4.441311738297539
time after is 2.2206558691487697
rehab
time before is 430.98062077295873
time after is 215.49031038647936
rehab
time before is 8.00955707100768
time after is 4.00477853550384
rehab

time before is 71.68950057732732
time after is 26.883562716497746
rehab
time before is 89.57553524115714
time after is 22.393883810289285
rehab
time before is 71.68950057732732
time after is 26.883562716497746
rehab
time before is 6.634003558693321
time after is 1.6585008896733302
rehab
time before is 3.0593642427709047
time after is 0.7648410606927262
rehab
time before is 38.36382722280522
time after is 9.590956805701305
rehab
time before is 6.634003558693321
time after is 1.6585008896733302
rehab
time before is 114.69478037688313
time after is 28.673695094220783
rehab
time before is 38.36382722280522
time after is 9.590956805701305
rehab
time before is 11.03923955922588
time after is 5.51961977961294
rehab
time before is 228.41981378145613
time after is 114.20990689072806
rehab
time before is 34.30636118700786
time after is 17.15318059350393
rehab
time before is 11.03923955922588
time after is 5.51961977961294
rehab
time before is 7.077726176231881
time after is 3.5388630881159404
re

time after is 46.8828653578191
rehab
time before is 224.7868778256089
time after is 98.3442590487039
rehab
time before is 130.78869424396177
time after is 57.22005373173327
rehab
time before is 191.9291760399067
time after is 47.98229400997668
rehab
time before is 121.96196437277786
time after is 30.490491093194464
rehab
time before is 191.9291760399067
time after is 47.98229400997668
rehab
time before is 100.71620037165833
time after is 25.179050092914583
rehab
time before is 121.96196437277786
time after is 30.490491093194464
rehab
time before is 76.59860500572003
time after is 19.149651251430008
rehab
time before is 100.71620037165833
time after is 25.179050092914583
rehab
time before is 137.57626305182208
time after is 34.39406576295552
rehab
time before is 76.59860500572003
time after is 19.149651251430008
rehab
time before is 200.6096454044633
time after is 50.15241135111582
rehab
time before is 137.57626305182208
time after is 34.39406576295552
rehab
time before is 46.1673703208

rehab
time before is 22.10266999397145
time after is 5.525667498492862
rehab
time before is 190.98114722646963
time after is 47.74528680661741
rehab
time before is 58.92172335947551
time after is 14.730430839868877
rehab
time before is 39.5159261516452
time after is 9.8789815379113
rehab
time before is 190.98114722646963
time after is 47.74528680661741
rehab
time before is 47.157663224050275
time after is 11.789415806012569
rehab
time before is 39.5159261516452
time after is 9.8789815379113
rehab
time before is 108.7723279666225
time after is 27.193081991655625
rehab
time before is 47.157663224050275
time after is 11.789415806012569
rehab
time before is 143.11042765859116
time after is 35.77760691464779
rehab
time before is 108.7723279666225
time after is 27.193081991655625
rehab
time before is 180.80060484894253
time after is 45.20015121223563
rehab
time before is 4.486519209593142
time after is 1.1216298023982856
rehab
time before is 37.59719323136074
time after is 9.399298307840185


time after is 23.545612500201813
rehab
time before is 24.34766646771425
time after is 9.130374925392847
rehab
time before is 110.20638667899934
time after is 41.327395004624755
rehab
time before is 62.78830000053817
time after is 23.545612500201813
rehab
time before is 212.93938361283142
time after is 79.85226885481178
rehab
time before is 110.20638667899934
time after is 41.327395004624755
rehab
time before is 5.69195405921255
time after is 2.1344827722047066
rehab
time before is 72.1713753067853
time after is 27.064265740044483
rehab
time before is 8.518740149561957
time after is 3.1945275560857342
rehab
time before is 5.69195405921255
time after is 2.1344827722047066
rehab
time before is 2.759697679606828
time after is 1.0348866298525607
rehab
time before is 8.518740149561957
time after is 3.1945275560857342
rehab
time before is 52.81564747609311
time after is 19.805867803534916
rehab
time before is 2.759697679606828
time after is 1.0348866298525607
rehab
time before is 42.803252634

# Import Destinations

In [21]:
destinations = gpd.read_file(r'input/HeRAMS_2018_April_snapped_renamed.shp')
#destinations = gpd.read_file(r'input/yemen_data_clusters_inland3.shp')
#destinations = gpd.read_file(r'input/district_pts.shp')

### filter destinations within the correct UTM zone

In [22]:
# UTM zone 38
destinations = destinations.cx[42:48, 0:90]
# UTM zone 39
#destinations = destinations.cx[48:54, 0:90]

## Is it possible to snap to adv graph but not to POI nodes?
Yes! use the adv graph, convert it to a point_gdf, then delete all of the POI nodes, then use the point GDF to do snapping!

This will result in more precise snapping.

Also, this takes care of the following edge case:
During the adv graph creation, I had a POI lie directly on top of an existing node. Therefore the POI node replaced the existing node on the graph. Then during the snapping of destinations to the original graph, one of the destinations snapped to the node used in the original graph, but was no longer being connected to edges in the adv graph. Therefore, during the calculate_OD step this resulted in a failed path.

In [23]:
G_adv_pt_gdf = gn.node_gdf_from_graph(G_adv_w_time)
G_adv_poi_pts = G_adv_pt_gdf.loc[G_adv_pt_gdf['infra_type'] != 'poi']

In [24]:
snapped_destinations_df = gn.pandana_snap_c(G_adv_poi_pts, destinations, source_crs='epsg:4326', target_crs=measure_crs)

In [25]:
snapped_destinations_df[:3]

,field_1,FID,Functional,YEHNP_PHCs,YEHNP_Hosp,Name of_H,Latitude,Longitude,name_of,Name of_1,...,Under-5 cl,Malnutri_1,Antenatal,Skilled _1,Basic Em_1,Emergenc_2,Comprehe_1,geometry,NN,NN_dist
0,0,1101001,2.000,NaN,NaN,Saihan HC,14.349,44.129,Ibb,Al Qafr,...,1.000,1.000,1.000,1.000,0.000,0.000,0.000,POINT (44.12940 14.34938),120278,129.411
1,1,1101002,1.000,1.000,NaN,Alsahlh H U,14.340,44.107,Ibb,Al Qafr,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,POINT (44.10663 14.34032),9990003706,621.769
2,2,1101003,2.000,NaN,NaN,Hadpan HU,14.337,44.154,Ibb,Al Qafr,...,0.000,1.000,1.000,1.000,0.000,0.000,0.000,POINT (44.15417 14.33722),127845,884.077


### Export to pickle to be used when calculating OD Matrix using graph-tool
CSVs don’t have types, that information is never preserved. CSVs only have strings. This is why we are saving as a pickle. Some of the destinations are strings, and some are ints.

In [26]:
snapped_destinations_df[['NN']].to_pickle('input2/destinations_health_38N.pickle')

In [27]:
destinationNodes = list(snapped_destinations_df['NN'].unique())

In [28]:
destinationNodes

['120278',
 9990003706,
 '127845',
 '111682',
 9990020760,
 'new_obj_1549',
 9990006603,
 '146191',
 9990025071,
 9990031804,
 9990006611,
 9990003690,
 '92862',
 9990032652,
 9990006711,
 9990028766,
 9990031359,
 9990020735,
 '1138',
 9990025695,
 9990006505,
 9990031041,
 9990018357,
 '129882',
 9990030981,
 9990032667,
 9990018696,
 9990019305,
 9990005322,
 9990026846,
 9990005385,
 9990006663,
 9990003676,
 9990038067,
 'new_obj_9531',
 '95916',
 '136247',
 '107245',
 '157840',
 9990005344,
 9990006445,
 9990005306,
 9990006571,
 9990006641,
 9990004060,
 '68259',
 9990026878,
 9990006737,
 9990037221,
 9990005355,
 '125278',
 9990002869,
 9990002838,
 9990002874,
 '161395',
 9990020407,
 9990026535,
 '170507',
 9990005407,
 9990020344,
 9990026573,
 9990002829,
 9990002867,
 9990037780,
 9990024708,
 9990040678,
 9990038850,
 'new_obj_9606',
 9990035872,
 9990024425,
 '74935',
 '86726',
 9990024668,
 9990021326,
 9990035970,
 9990001592,
 '103590',
 9990003576,
 9990005540,
 999

In [29]:
len(destinationNodes)

3649

In [30]:
if 9990018860 in destinationNodes:
    print('yes')

## Export Graphs to be run in Calculate OD using graph-tool

In [31]:
G_adv_w_time_export = G_adv_w_time.copy()

In [32]:
for n1, n2, data in G_adv_w_time_export.edges(data=True):
    #print(data)
    #del data['infra_type']
    #del data['Type']
    #del data['mode']
    del data['key']
    del data['name']
    del data['id']

In [33]:
for node, data in G_adv_w_time_export.nodes(data=True):
    #print(data)
    del data['infra_type']

In [34]:
G_adv_edges2 = gn.edge_gdf_from_graph(G_adv_w_time_export)
G_adv_edges2[:3]

,stnode,endnode,osm_id,one_way,infra_type,Type,mode,time,length,geometry
0,1,110147,810908760,False,unclassified,legitimate,drive,11.022,61.234,"LINESTRING (44.18929 15.45383, 44.18931 15.45438)"
1,1,144649,810908766,False,unclassified,legitimate,drive,27.986,155.478,"LINESTRING (44.18929 15.45383, 44.18784 15.45386)"
2,1,21643,810908760,False,unclassified,legitimate,drive,11.122,61.787,"LINESTRING (44.18927 15.45327, 44.18929 15.45383)"


In [35]:
G_adv_nodes2 = gn.node_gdf_from_graph(G_adv_w_time_export)
G_adv_nodes2[:3]

,node_ID,x,y,geometry
0,1,44.189,15.454,POINT (44.18929 15.45383)
1,110147,44.189,15.454,POINT (44.18931 15.45438)
2,144649,44.188,15.454,POINT (44.18784 15.45386)


In [36]:
gn.save(G_adv_w_time_export, 'G_adv_w_time', 'input2', pickle=True, edges=True, nodes=True)

In [37]:
G_adv_w_time_rehab_export = G_adv_w_time_rehab.copy()

In [38]:
for n1, n2, data in G_adv_w_time_rehab_export.edges(data=True):
    #print(data)
    del data['infra_type']
    del data['Type']
    del data['mode']
    del data['key']
    del data['name']
    del data['id']

In [39]:
for node, data in G_adv_w_time_rehab_export.nodes(data=True):
    #print(data)
    del data['infra_type']

In [40]:
G_adv_edges3 = gn.edge_gdf_from_graph(G_adv_w_time_rehab_export)
G_adv_edges3

,stnode,endnode,osm_id,one_way,time,length,geometry
0,1,110147,810908760,False,11.022,61.234,"LINESTRING (44.18929 15.45383, 44.18931 15.45438)"
1,1,144649,810908766,False,27.986,155.478,"LINESTRING (44.18929 15.45383, 44.18784 15.45386)"
2,1,21643,810908760,False,11.122,61.787,"LINESTRING (44.18927 15.45327, 44.18929 15.45383)"
3,110147,96034,810908760,False,51.482,286.013,"LINESTRING (44.18931 15.45438, 44.18933 15.454..."
4,110147,46500,810908765,False,28.098,156.100,"LINESTRING (44.18785 15.45441, 44.18931 15.45438)"
...,...,...,...,...,...,...,...
529092,8941401868.000,9990049963,9990049963_8941401868.0,True,54.567,53.052,"LINESTRING (44.14249 14.58104, 44.14201 14.58115)"
529093,8996326622.000,128985,128985_8996326622.0,True,116.481,113.245,"LINESTRING (44.88386 13.01905, 44.88481 13.01949)"
529094,8996697205.000,31124,31124_8996697205.0,True,140.164,136.270,"LINESTRING (44.88357 13.01881, 44.88479 13.01850)"
529095,8999185829.000,31124,31124_8999185829.0,True,129.397,125.802,"LINESTRING (44.88363 13.01848, 44.88479 13.01850)"


In [41]:
G_adv_nodes3 = gn.node_gdf_from_graph(G_adv_w_time_rehab_export)
G_adv_nodes3

,node_ID,x,y,geometry
0,1,44.189,15.454,POINT (44.18929 15.45383)
1,110147,44.189,15.454,POINT (44.18931 15.45438)
2,144649,44.188,15.454,POINT (44.18784 15.45386)
3,21643,44.189,15.453,POINT (44.18927 15.45327)
4,96034,44.187,15.455,POINT (44.18727 15.45504)
...,...,...,...,...
271304,8941401868.000,44.142,14.581,POINT (44.14249 14.58104)
271305,8996326622.000,44.884,13.019,POINT (44.88386 13.01905)
271306,8996697205.000,44.884,13.019,POINT (44.88357 13.01881)
271307,8999185829.000,44.884,13.018,POINT (44.88363 13.01848)


In [42]:
gn.save(G_adv_w_time_rehab_export, 'G_adv_w_time_rehab', 'input2', pickle=True, edges=True, nodes=True)

## Calculate OD matrix for regular graph

In [105]:
%%time
#OD_matrix = gn.calculate_OD(G_adv_w_time, originNodes, destinationNodes, fail_value=-1, weight='time', one_way_roads_exist = True)

CPU times: user 9 µs, sys: 0 ns, total: 9 µs
Wall time: 17.4 µs


In [106]:
# load OD matrix from CSV
OD_matrix = pd.read_pickle(r'output2/OD_matrix_health_38N.pickle')

In [107]:
OD_matrix

,9990029313,9990012935,9990012936,59634,9990012937,69714,9990021142,64523,9990029343,9990012961,...,9990037485,100460,104267,90553,9990004713,9990004714,9990004730,81900,9990021116,9990012926
1219231756.000,32734.199,37579.981,32722.848,20758.730,34338.529,14903.395,24761.949,12314.201,36052.254,35443.917,...,23912.963,75083.948,28446.287,40931.806,9039.715,8766.399,8888.614,46077.365,32900.273,27486.929
1219231762.000,31863.014,37496.402,31851.663,20712.374,34292.173,14032.210,24715.594,11443.016,35181.069,34572.732,...,23041.778,74212.763,27575.102,40060.621,8993.359,8720.044,8842.259,45206.180,32029.089,26615.744
1219231767.000,31617.708,37251.096,31606.357,20967.714,34547.513,13786.903,24970.934,11197.710,34935.763,34327.426,...,22796.471,73967.457,27329.796,39815.315,9248.699,8975.384,9097.599,44960.874,31783.782,26370.437
1219231772.000,31937.069,36782.851,31925.718,19961.600,33541.399,14106.265,23964.820,11517.071,35255.124,34646.787,...,23115.833,74286.818,27649.157,40134.676,8242.585,7969.270,8091.485,45280.235,32103.143,26689.799
1219231819.000,34288.606,39921.994,34277.255,23638.612,37218.411,16457.801,27641.832,13868.608,37606.661,36998.324,...,25467.369,76638.354,30000.693,42486.213,11919.597,11646.281,11768.497,47631.772,34454.680,29041.335
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6936545145.000,8581.835,46189.174,40544.435,31528.928,44736.693,23825.658,38248.763,34201.587,7303.705,43265.504,...,18456.321,81494.630,11293.276,17312.011,35450.306,35176.990,35299.205,28691.122,9280.478,35308.516
6936545146.000,8627.473,46234.812,40590.073,31574.566,44782.331,23871.296,38294.401,34247.225,7349.343,43311.142,...,18501.959,81540.268,11338.914,17357.649,35495.943,35222.628,35344.843,28736.760,9326.116,35354.154
5699796921.000,16619.774,35275.828,29631.089,20615.582,33823.346,19397.447,27335.417,20407.113,15203.011,32352.158,...,13358.968,70581.283,16134.143,28784.251,22442.995,22169.679,22291.894,25600.466,20752.719,24395.169
6936545162.000,7895.343,46244.423,40599.684,31584.177,44791.942,23139.167,38304.013,33515.095,6840.084,43320.753,...,17769.829,81549.879,10606.784,17367.260,34763.814,34490.498,34612.713,28746.371,9335.728,35363.765


In [108]:
OD_matrix.min(axis=1)

1219231756.000   3276.793
1219231762.000   4009.797
1219231767.000   3372.802
1219231772.000   2479.663
1219231819.000   5590.075
                   ...   
6936545145.000    313.878
6936545146.000    153.464
5699796921.000   1182.512
6936545162.000    681.431
6936545163.000    385.570
Length: 79334, dtype: float64

In [109]:
colName = "travel_time_to_closest_facility"
results = pd.DataFrame({'osm_id':OD_matrix.min(axis=1).index, colName:OD_matrix.min(axis=1).values})
results

,osm_id,travel_time_to_closest_facility
0,1219231756.000,3276.793
1,1219231762.000,4009.797
2,1219231767.000,3372.802
3,1219231772.000,2479.663
4,1219231819.000,5590.075
...,...,...
79329,6936545145.000,313.878
79330,6936545146.000,153.464
79331,5699796921.000,1182.512
79332,6936545162.000,681.431


In [110]:
# test
results[results['travel_time_to_closest_facility']==-1]

,osm_id,travel_time_to_closest_facility


In [111]:
output2 = pd.merge(pois_meter, results, on="osm_id")

In [112]:
output2[["travel_time_to_closest_facility"]] = output2[["travel_time_to_closest_facility"]].apply(pd.to_numeric)

In [113]:
output2 = output2.drop(columns=['near_idx','near_lines','pp'], axis=1)

In [114]:
output2['travel_time_to_closest_facility'].mean()

1778.1937578092673

In [115]:
output2

,osm_id,population,place,name,source,is_in,geometry,kne_idx,travel_time_to_closest_facility
0,258013552.000,1527861,city,صنعاء,None,None,POINT (44.20588 15.35386),181104,152.133
1,263261532.000,458933,city,تعز,None,None,POINT (44.02153 13.57522),76936,48.297
2,263263790.000,508489,city,عدن,None,None,POINT (45.02850 12.78959),69025,351.477
3,268256784.000,208844,city,إب,None,None,POINT (44.17794 13.97016),81160,181.387
4,296512221.000,144273,city,ذمار,None,None,POINT (44.40006 14.54300),9472,39.282
...,...,...,...,...,...,...,...,...,...
79329,8941401868.000,261,village,وادي الامحال,None,None,POINT (44.14249 14.58104),171125,1683.356
79330,8996326622.000,None,town,None,None,None,POINT (44.88386 13.01905),84302,559.530
79331,8996697205.000,None,town,None,None,None,POINT (44.88357 13.01881),87578,629.847
79332,8999185829.000,None,town,None,None,None,POINT (44.88363 13.01848),87578,619.080


In [116]:
reg_accessibilty_gpd = gpd.GeoDataFrame(output2, crs = "epsg:4326", geometry = 'geometry')
#reg_accessibilty_gpd.to_file('yemen_accessibility_district_centers_wo_walking_time2.shp')
#reg_accessibilty_gpd.to_file('yemen_38n_accessibility_district_centers_w_walking_time3.shp')
reg_accessibilty_gpd.to_file('output2/yemen_38n_accessibility_health_w_walking_time_6.shp')

<ipython-input-116-152bae46cb37>:4: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  reg_accessibilty_gpd.to_file('output2/yemen_38n_accessibility_health_w_walking_time_6.shp')


## Calculate OD matrix for graph with rehabilitated roads

In [117]:
#OD_matrix2 = gn.calculate_OD(G_adv_w_time_rehab, originNodes, destinationNodes, fail_value=-1, weight='time', one_way_roads_exist = True)

In [118]:
# load OD matrix from CSV
OD_matrix2 = pd.read_pickle(r'output2/OD_matrix_health_38N_rehab.pickle')

In [119]:
%%time
# closest_facility_per_origin2 = OD_matrix2.min(axis=1)
# results2 = pd.DataFrame([originNodes, closest_facility_per_origin2]).transpose()
# colName = "travel_time_to_closest_facility"
# results2.columns = ['osm_id', colName]

colName = "travel_time_to_closest_facility"
results2 = pd.DataFrame({'osm_id':OD_matrix2.min(axis=1).index, colName:OD_matrix2.min(axis=1).values})
results2

CPU times: user 859 ms, sys: 70.6 ms, total: 929 ms
Wall time: 928 ms


,osm_id,travel_time_to_closest_facility
0,1219231756.000,3276.793
1,1219231762.000,4009.797
2,1219231767.000,3372.802
3,1219231772.000,2479.663
4,1219231819.000,5590.075
...,...,...
79329,6936545145.000,313.878
79330,6936545146.000,153.464
79331,5699796921.000,1182.512
79332,6936545162.000,681.431


In [120]:
output3 = pd.merge(pois_meter, results2, on="osm_id")
output3[["travel_time_to_closest_facility"]] = output3[["travel_time_to_closest_facility"]].apply(pd.to_numeric)

In [121]:
output3['travel_time_to_closest_facility'].mean()

1772.7479443832847

In [122]:
output3 = output3.drop(columns=['near_idx','near_lines','pp'], axis=1)

In [123]:
accessibilty_gpd = gpd.GeoDataFrame(output3, crs = "epsg:4326", geometry = 'geometry')
#accessibilty_gpd.to_file('yemen_accessibility_district_centers_rehabbed_wo_walking_time2.shp')
#accessibilty_gpd.to_file('yemen_38n_accessibility_district_centers_rehabbed_w_walking_time3.shp')
accessibilty_gpd.to_file('output2/yemen_38n_accessibility_health_w_walking_time_rehab_6.shp')

<ipython-input-123-c993fb5e7047>:4: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  accessibilty_gpd.to_file('output2/yemen_38n_accessibility_health_w_walking_time_rehab_6.shp')


## Calculate average travel time per Rehabilitated road

In [124]:
origins_to_closest_facility_gpd = gpd.GeoDataFrame(output2, crs = "epsg:4326", geometry = 'geometry')

In [125]:
origins_to_closest_facility_gpd

,osm_id,population,place,name,source,is_in,geometry,kne_idx,travel_time_to_closest_facility
0,258013552.000,1527861,city,صنعاء,None,None,POINT (44.20588 15.35386),181104,152.133
1,263261532.000,458933,city,تعز,None,None,POINT (44.02153 13.57522),76936,48.297
2,263263790.000,508489,city,عدن,None,None,POINT (45.02850 12.78959),69025,351.477
3,268256784.000,208844,city,إب,None,None,POINT (44.17794 13.97016),81160,181.387
4,296512221.000,144273,city,ذمار,None,None,POINT (44.40006 14.54300),9472,39.282
...,...,...,...,...,...,...,...,...,...
79329,8941401868.000,261,village,وادي الامحال,None,None,POINT (44.14249 14.58104),171125,1683.356
79330,8996326622.000,None,town,None,None,None,POINT (44.88386 13.01905),84302,559.530
79331,8996697205.000,None,town,None,None,None,POINT (44.88357 13.01881),87578,629.847
79332,8999185829.000,None,town,None,None,None,POINT (44.88363 13.01848),87578,619.080


In [126]:
origins_to_closest_facility_gpd_prj = origins_to_closest_facility_gpd.to_crs(measure_crs)

### Read fille of 5k buffers around roads
QGIS steps: select all rehab roads, save a 38N projection, buffer, dissolve, then multi-part to single-part

In [127]:
rehab_rds_yemen_38n_5k_buffer = gpd.read_file(r'rehab_rds_only_G_largest_cleaned_yemen4_single_edge2_38n_5k_buffer5.shp')

In [128]:
rehab_rds_yemen_38n_5k_buffer

,stnode,endnode,id,one_way,osm_id,Type,infra_type,key,length,name,geometry
0,170,147350,91987,0,436184635,rehab,trunk,edge_91987,0.978,al_abr_to_seiyun,"POLYGON ((423850.447 1450694.145, 423836.637 1..."
1,170,147350,91987,0,436184635,rehab,trunk,edge_91987,0.978,al_abr_to_seiyun,"POLYGON ((372427.033 1512304.025, 372581.704 1..."
2,170,147350,91987,0,436184635,rehab,trunk,edge_91987,0.978,al_abr_to_seiyun,"POLYGON ((402505.912 1607646.041, 402493.142 1..."
3,170,147350,91987,0,436184635,rehab,trunk,edge_91987,0.978,al_abr_to_seiyun,"POLYGON ((417612.078 1548291.364, 418798.561 1..."
4,170,147350,91987,0,436184635,rehab,trunk,edge_91987,0.978,al_abr_to_seiyun,"POLYGON ((416288.856 1561210.961, 415879.644 1..."
5,170,147350,91987,0,436184635,rehab,trunk,edge_91987,0.978,al_abr_to_seiyun,"POLYGON ((502841.446 1575046.924, 501576.664 1..."
6,170,147350,91987,0,436184635,rehab,trunk,edge_91987,0.978,al_abr_to_seiyun,"POLYGON ((468920.058 1584746.367, 468703.966 1..."
7,170,147350,91987,0,436184635,rehab,trunk,edge_91987,0.978,al_abr_to_seiyun,"POLYGON ((542731.058 1447148.254, 542648.482 1..."
8,170,147350,91987,0,436184635,rehab,trunk,edge_91987,0.978,al_abr_to_seiyun,"POLYGON ((485036.171 1506022.583, 485086.685 1..."
9,170,147350,91987,0,436184635,rehab,trunk,edge_91987,0.978,al_abr_to_seiyun,"POLYGON ((471249.443 1501611.368, 469790.402 1..."


In [129]:
rehab_rds_yemen_38n_5k_buffer.crs

<Derived Projected CRS: EPSG:32638>
Name: WGS 84 / UTM zone 38N
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Between 42°E and 48°E, northern hemisphere between equator and 84°N, onshore and offshore. Armenia. Azerbaijan. Djibouti. Eritrea. Ethiopia. Georgia. Islamic Republic of Iran. Iraq. kazakhstan. Kuwait. Russian Federation. Saudi Arabia. Somalia. Turkey. Yemen.
- bounds: (42.0, 0.0, 48.0, 84.0)
Coordinate Operation:
- name: UTM zone 38N
- method: Transverse Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [130]:
rehab_rds_yemen_38n_5k_buffer.iloc[[0]]

,stnode,endnode,id,one_way,osm_id,Type,infra_type,key,length,name,geometry
0,170,147350,91987,0,436184635,rehab,trunk,edge_91987,0.978,al_abr_to_seiyun,"POLYGON ((423850.447 1450694.145, 423836.637 1..."


In [131]:
points_within = gpd.sjoin(origins_to_closest_facility_gpd_prj, rehab_rds_yemen_38n_5k_buffer, op='within')

/opt/conda/envs/geo_env/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3377: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if (await self.run_code(code, result,  async_=asy)):
<ipython-input-131-eea28ffc0127>:1: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:2089
Right CRS: EPSG:32638

  points_within = gpd.sjoin(origins_to_closest_facility_gpd_prj, rehab_rds_yemen_38n_5k_buffer, op='within')


In [132]:
points_within

,osm_id_left,population,place,name_left,source,is_in,geometry,kne_idx,travel_time_to_closest_facility,index_right,stnode,endnode,id,one_way,osm_id_right,Type,infra_type,key,length,name_right
1,263261532.000,458933,city,تعز,None,None,POINT (394138.111 1500961.213),76936,48.297,0,170,147350,91987,0,436184635,rehab,trunk,edge_91987,0.978,al_abr_to_seiyun
32,1216827057.000,170,hamlet,النجد,None,None,POINT (427254.748 1452614.479),17583,414.043,0,170,147350,91987,0,436184635,rehab,trunk,edge_91987,0.978,al_abr_to_seiyun
3214,1219305839.000,108,hamlet,قرية الدار,None,None,POINT (386005.958 1496432.056),143245,336.258,0,170,147350,91987,0,436184635,rehab,trunk,edge_91987,0.978,al_abr_to_seiyun
3215,1219305844.000,81,hamlet,الكريفة,None,None,POINT (381276.675 1497180.431),72140,2694.413,0,170,147350,91987,0,436184635,rehab,trunk,edge_91987,0.978,al_abr_to_seiyun
3216,1219305850.000,111,hamlet,شعب العكاري,None,None,POINT (382131.260 1500861.181),95237,1571.640,0,170,147350,91987,0,436184635,rehab,trunk,edge_91987,0.978,al_abr_to_seiyun
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61839,6936988504.000,None,hamlet,المشرقي,None,None,POINT (526893.454 1661136.746),17134,3075.044,13,170,147350,91987,0,436184635,rehab,trunk,edge_91987,0.978,al_abr_to_seiyun
62036,6936988703.000,11,hamlet,الكتف,None,None,POINT (524498.331 1669565.424),179783,3315.173,13,170,147350,91987,0,436184635,rehab,trunk,edge_91987,0.978,al_abr_to_seiyun
62101,6936988768.000,None,hamlet,ذراع العطيف,None,None,POINT (521608.007 1660419.052),101459,155.156,13,170,147350,91987,0,436184635,rehab,trunk,edge_91987,0.978,al_abr_to_seiyun
62296,6936988963.000,None,hamlet,الريام,None,None,POINT (522783.917 1667152.607),57586,2101.252,13,170,147350,91987,0,436184635,rehab,trunk,edge_91987,0.978,al_abr_to_seiyun


In [133]:
list(points_within['osm_id_left'])

[263261532.0,
 1216827057.0,
 1219305839.0,
 1219305844.0,
 1219305850.0,
 1219305853.0,
 1219305862.0,
 1219305888.0,
 1219305896.0,
 1219305906.0,
 1219305927.0,
 1219305933.0,
 1219305958.0,
 1219305995.0,
 1219305998.0,
 1219306016.0,
 1219306017.0,
 1219306019.0,
 1219306034.0,
 1219306038.0,
 1219306120.0,
 1219306141.0,
 1219306149.0,
 1219306153.0,
 1219306164.0,
 1219306168.0,
 1219306172.0,
 1219306191.0,
 1219306858.0,
 1219306868.0,
 1219306871.0,
 1219306909.0,
 1219306975.0,
 1219307014.0,
 1219307018.0,
 1219307037.0,
 1219307062.0,
 1219307076.0,
 1219307081.0,
 1219307101.0,
 1219307134.0,
 1219307206.0,
 1219307225.0,
 1219307255.0,
 1219307285.0,
 1219307336.0,
 1219307450.0,
 1219307578.0,
 1219398896.0,
 1219398925.0,
 1219398962.0,
 1219398994.0,
 1219399038.0,
 1219399234.0,
 1219399431.0,
 1219399445.0,
 1219399470.0,
 1219399619.0,
 1219399632.0,
 1219399693.0,
 1219399801.0,
 1219399972.0,
 1219400060.0,
 1219400098.0,
 1219401981.0,
 1219401997.0,
 1219403426

In [134]:
OD_matrix2

,new_obj_253,9990029313,71983,11443,9990012935,9990012936,9999007,9990012937,128143,170229,...,9990004713,88384,109978,9990037485,28053,9990004714,10517,9990004730,9990021116,9990012926
1219231756.000,37450.278,30148.095,40037.580,36544.167,37579.981,32722.848,34185.905,34338.529,26027.734,28257.651,...,9039.715,40970.544,29515.685,23539.219,36315.279,8766.399,46426.106,8888.614,32900.273,27486.929
1219231762.000,36579.093,29276.910,39166.395,35672.982,37496.402,31851.663,33314.720,34292.173,25156.549,27386.466,...,8993.359,40099.359,28644.500,22668.034,35444.095,8720.044,45554.921,8842.259,32029.089,26615.744
1219231767.000,36333.787,29031.604,38921.089,35427.675,37251.096,31606.357,33069.414,34547.513,24911.243,27141.160,...,9248.699,39854.053,28399.193,22422.727,35198.788,8975.384,45309.614,9097.599,31783.782,26370.437
1219231772.000,36653.148,29350.965,39240.450,35747.037,36782.851,31925.718,33388.775,33541.399,25230.604,27460.521,...,8242.585,40173.414,28718.555,22742.089,35518.149,7969.270,45628.976,8091.485,32103.143,26689.799
1219231819.000,39004.685,31702.502,41591.987,38098.573,39921.994,34277.255,35740.312,37218.411,27582.141,29812.058,...,11919.597,42524.951,31070.091,25093.625,37869.686,11646.281,47980.512,11768.497,34454.680,29041.335
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6936545145.000,5544.274,6618.041,29524.200,5965.599,46189.174,40544.435,21910.032,44736.693,10039.355,17917.760,...,35450.306,30457.164,20296.263,18395.931,10295.894,35176.990,32271.151,35299.205,9280.478,35308.516
6936545146.000,5589.912,6663.679,29569.838,6011.236,46234.812,40590.073,21955.669,44782.331,10084.993,17963.398,...,35495.943,30502.802,20341.901,18441.569,10341.531,35222.628,32316.789,35344.843,9326.116,35354.154
5699796921.000,16419.326,14095.994,19560.681,13799.589,35275.828,29631.089,13709.006,33823.346,16057.642,7780.752,...,22442.995,20493.645,6847.381,13358.968,9201.016,22169.679,25949.206,22291.894,20752.719,24395.169
6936545162.000,5599.523,5931.549,29579.449,6020.848,46244.423,40599.684,21965.281,44791.942,9352.863,17973.009,...,34763.814,30512.413,20351.512,17709.439,10351.143,34490.498,32326.400,34612.713,9335.728,35363.765


In [135]:
list(OD_matrix2.min(axis=1))

[3276.792818370697,
 4009.79712407062,
 3372.8019777544605,
 2479.6629812065125,
 5590.075177854306,
 2118.6207550662502,
 3970.0071087331935,
 1384.5913612028905,
 4542.703789525023,
 4905.954163836188,
 2461.514612059287,
 5627.011044015872,
 3012.802215745589,
 1824.5768852280887,
 502.9283329585619,
 1816.3625817974091,
 2759.534608733899,
 1550.4524796441422,
 3416.3730350984156,
 3863.018422715488,
 1395.255389895724,
 1206.6714197348308,
 5329.532813551344,
 2080.992211900892,
 138.8325437879703,
 1785.915527673768,
 927.8606649137648,
 1086.4244577870677,
 812.9994371507894,
 1042.2966297908872,
 566.5582500364516,
 767.4778978354963,
 476.67432804600315,
 1713.6678499678358,
 260.9077815857501,
 724.6434757619508,
 644.4522284974613,
 1022.3376697856735,
 448.58236442995894,
 2002.3163544343115,
 960.6433251093257,
 396.087080513413,
 697.9437721736244,
 197.37749287174663,
 964.846123306625,
 420.37217942298975,
 6418.175776477371,
 1258.5508413053403,
 4470.711550294905,
 27

In [136]:
OD_df_rehab = OD_matrix2

In [137]:
closest_facility_id_per_origin_rehab = OD_matrix2.idxmin(axis=1)

In [138]:
closest_facility_id_per_origin_rehab

1219231756.000    9990002227
1219231762.000    9990013680
1219231767.000    9990013680
1219231772.000    9990002227
1219231819.000    9990013680
                     ...    
6936545145.000    9990034821
6936545146.000          8056
5699796921.000    9990036266
6936545162.000    9990023001
6936545163.000    9990023001
Length: 79334, dtype: object

In [139]:
closest_facility_id_per_origin_rehab2 = closest_facility_id_per_origin_rehab[closest_facility_id_per_origin_rehab.index.isin(list(points_within['osm_id_left']))]
closest_facility_id_per_origin_rehab2

6936545281.000    9990036989
6936545287.000    9990018857
1217921704.000    9990006478
1226310835.000    9990002723
1226311149.000    9990002723
                     ...    
1217920540.000    9990049274
1217920582.000        133916
1217920644.000        133916
6936545101.000    9990028147
7294960930.000    9990008762
Length: 10090, dtype: object

In [140]:
list(closest_facility_id_per_origin_rehab2)

[9990036989,
 9990018857,
 9990006478,
 9990002723,
 9990002723,
 9990002723,
 9990010994,
 9990002723,
 9990002723,
 9990002723,
 9990011008,
 '37092',
 9990011008,
 9990002723,
 9990011008,
 '37092',
 9990011008,
 9990046317,
 9990011008,
 '124358',
 9990011008,
 9990011008,
 '99213',
 '99213',
 '99213',
 9990040896,
 9990040896,
 9990040896,
 '68408',
 9990040896,
 9990031342,
 9990031342,
 9990006955,
 9990040896,
 9990015563,
 9990034979,
 9990005858,
 9990010413,
 9990047436,
 9990010413,
 9990010413,
 9990010413,
 9990010413,
 9990011008,
 '37092',
 '37092',
 9990011008,
 '4869',
 9990023554,
 '94755',
 '163242',
 9990045023,
 9990049243,
 9990044504,
 '163242',
 9990044697,
 9990043958,
 '109871',
 9990048834,
 9990006432,
 '141431',
 9990044504,
 '4869',
 9990001679,
 9990000576,
 '97056',
 9990011339,
 '61438',
 '94755',
 9990044383,
 9990043690,
 9990044698,
 9990046777,
 'new_obj_263',
 9990040624,
 9990040180,
 9990045744,
 '4326',
 '4326',
 '141431',
 9990000241,
 '4326',

In [141]:
closest_facility_id_per_origin_rehab2

6936545281.000    9990036989
6936545287.000    9990018857
1217921704.000    9990006478
1226310835.000    9990002723
1226311149.000    9990002723
                     ...    
1217920540.000    9990049274
1217920582.000        133916
1217920644.000        133916
6936545101.000    9990028147
7294960930.000    9990008762
Length: 10090, dtype: object

In [142]:
OD_df_reg = OD_matrix

In [143]:
OD_matrix

,9990029313,9990012935,9990012936,59634,9990012937,69714,9990021142,64523,9990029343,9990012961,...,9990037485,100460,104267,90553,9990004713,9990004714,9990004730,81900,9990021116,9990012926
1219231756.000,32734.199,37579.981,32722.848,20758.730,34338.529,14903.395,24761.949,12314.201,36052.254,35443.917,...,23912.963,75083.948,28446.287,40931.806,9039.715,8766.399,8888.614,46077.365,32900.273,27486.929
1219231762.000,31863.014,37496.402,31851.663,20712.374,34292.173,14032.210,24715.594,11443.016,35181.069,34572.732,...,23041.778,74212.763,27575.102,40060.621,8993.359,8720.044,8842.259,45206.180,32029.089,26615.744
1219231767.000,31617.708,37251.096,31606.357,20967.714,34547.513,13786.903,24970.934,11197.710,34935.763,34327.426,...,22796.471,73967.457,27329.796,39815.315,9248.699,8975.384,9097.599,44960.874,31783.782,26370.437
1219231772.000,31937.069,36782.851,31925.718,19961.600,33541.399,14106.265,23964.820,11517.071,35255.124,34646.787,...,23115.833,74286.818,27649.157,40134.676,8242.585,7969.270,8091.485,45280.235,32103.143,26689.799
1219231819.000,34288.606,39921.994,34277.255,23638.612,37218.411,16457.801,27641.832,13868.608,37606.661,36998.324,...,25467.369,76638.354,30000.693,42486.213,11919.597,11646.281,11768.497,47631.772,34454.680,29041.335
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6936545145.000,8581.835,46189.174,40544.435,31528.928,44736.693,23825.658,38248.763,34201.587,7303.705,43265.504,...,18456.321,81494.630,11293.276,17312.011,35450.306,35176.990,35299.205,28691.122,9280.478,35308.516
6936545146.000,8627.473,46234.812,40590.073,31574.566,44782.331,23871.296,38294.401,34247.225,7349.343,43311.142,...,18501.959,81540.268,11338.914,17357.649,35495.943,35222.628,35344.843,28736.760,9326.116,35354.154
5699796921.000,16619.774,35275.828,29631.089,20615.582,33823.346,19397.447,27335.417,20407.113,15203.011,32352.158,...,13358.968,70581.283,16134.143,28784.251,22442.995,22169.679,22291.894,25600.466,20752.719,24395.169
6936545162.000,7895.343,46244.423,40599.684,31584.177,44791.942,23139.167,38304.013,33515.095,6840.084,43320.753,...,17769.829,81549.879,10606.784,17367.260,34763.814,34490.498,34612.713,28746.371,9335.728,35363.765


In [144]:
OD_matrix2

,new_obj_253,9990029313,71983,11443,9990012935,9990012936,9999007,9990012937,128143,170229,...,9990004713,88384,109978,9990037485,28053,9990004714,10517,9990004730,9990021116,9990012926
1219231756.000,37450.278,30148.095,40037.580,36544.167,37579.981,32722.848,34185.905,34338.529,26027.734,28257.651,...,9039.715,40970.544,29515.685,23539.219,36315.279,8766.399,46426.106,8888.614,32900.273,27486.929
1219231762.000,36579.093,29276.910,39166.395,35672.982,37496.402,31851.663,33314.720,34292.173,25156.549,27386.466,...,8993.359,40099.359,28644.500,22668.034,35444.095,8720.044,45554.921,8842.259,32029.089,26615.744
1219231767.000,36333.787,29031.604,38921.089,35427.675,37251.096,31606.357,33069.414,34547.513,24911.243,27141.160,...,9248.699,39854.053,28399.193,22422.727,35198.788,8975.384,45309.614,9097.599,31783.782,26370.437
1219231772.000,36653.148,29350.965,39240.450,35747.037,36782.851,31925.718,33388.775,33541.399,25230.604,27460.521,...,8242.585,40173.414,28718.555,22742.089,35518.149,7969.270,45628.976,8091.485,32103.143,26689.799
1219231819.000,39004.685,31702.502,41591.987,38098.573,39921.994,34277.255,35740.312,37218.411,27582.141,29812.058,...,11919.597,42524.951,31070.091,25093.625,37869.686,11646.281,47980.512,11768.497,34454.680,29041.335
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6936545145.000,5544.274,6618.041,29524.200,5965.599,46189.174,40544.435,21910.032,44736.693,10039.355,17917.760,...,35450.306,30457.164,20296.263,18395.931,10295.894,35176.990,32271.151,35299.205,9280.478,35308.516
6936545146.000,5589.912,6663.679,29569.838,6011.236,46234.812,40590.073,21955.669,44782.331,10084.993,17963.398,...,35495.943,30502.802,20341.901,18441.569,10341.531,35222.628,32316.789,35344.843,9326.116,35354.154
5699796921.000,16419.326,14095.994,19560.681,13799.589,35275.828,29631.089,13709.006,33823.346,16057.642,7780.752,...,22442.995,20493.645,6847.381,13358.968,9201.016,22169.679,25949.206,22291.894,20752.719,24395.169
6936545162.000,5599.523,5931.549,29579.449,6020.848,46244.423,40599.684,21965.281,44791.942,9352.863,17973.009,...,34763.814,30512.413,20351.512,17709.439,10351.143,34490.498,32326.400,34612.713,9335.728,35363.765


In [145]:
from collections import defaultdict

In [146]:
def group(lst, n):
  for i in range(0, len(lst), 1):
    val = lst[i:i+n]
    if len(val) == n:
      yield tuple(val)

In [147]:
#https://stackoverflow.com/questions/26367812/appending-to-list-in-python-dictionary
rehab_road_tracker = defaultdict(list)

In [148]:
rehab_origin_destination_pairs = []

In [149]:
def look_up(shortest_path, shortest_path_time, row, col):
    shortest_path_rehab_road_tracker = []
    edge_segements = list(group(shortest_path,2))
    for edge in edge_segements:
        try:    
            if G_adv_w_time_rehab[edge[0]][edge[1]][0]['Type'] == 'rehab':
                #print('rehab match')
                road_name = G_adv_w_time_rehab[edge[0]][edge[1]][0]['name']
                shortest_path_rehab_road_tracker.append(road_name)
                #break
        except:
            print(G_adv_w_time_rehab[edge[0]][edge[1]][0])
            print(edge[0])
            print(edge[1])
            global mismatch_count
            mismatch_count += 1
    
    if len(shortest_path_rehab_road_tracker) > 0:
        
        shortest_path_rehab_road_tracker = list(set(shortest_path_rehab_road_tracker))
        
        if len(shortest_path_rehab_road_tracker) > 1:
            print('This shortest path traversed more than one rehab roads')
        
        print(f"shortest_path_rehab_road_tracker is: {shortest_path_rehab_road_tracker}")

        for rehab_road in shortest_path_rehab_road_tracker:
            print(f"will append road name: {rehab_road} with shortest path time: {shortest_path_time}")
            rehab_road_tracker[rehab_road].append(shortest_path_time)

        rehab_origin_destination_pairs.append((row, col))

In [150]:
%time
mismatch_count = 0
for row, col in closest_facility_id_per_origin_rehab2.items():
    #print(row)
    #print(col)
    shortest_path = nx.dijkstra_path(G_adv_w_time_rehab, row, col, weight='time')
    shortest_path_time = OD_df_rehab.loc[row, col]
    look_up(shortest_path, shortest_path_time, row, col)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 3.81 µs
shortest_path_rehab_road_tracker is: ['hajdah_rasian']
will append road name: hajdah_rasian with shortest path time: 1785.915527673768
shortest_path_rehab_road_tracker is: ['mnar_anes']
will append road name: mnar_anes with shortest path time: 2002.3163544343115
shortest_path_rehab_road_tracker is: ['sbarah_sanahi']
will append road name: sbarah_sanahi with shortest path time: 215.51746546520792
shortest_path_rehab_road_tracker is: ['sbarah_sanahi']
will append road name: sbarah_sanahi with shortest path time: 518.5970215816039
shortest_path_rehab_road_tracker is: ['sbarah_sanahi']
will append road name: sbarah_sanahi with shortest path time: 1072.2300084894919
shortest_path_rehab_road_tracker is: ['sbarah_sanahi']
will append road name: sbarah_sanahi with shortest path time: 1015.5262843238004
shortest_path_rehab_road_tracker is: ['sbarah_sanahi']
will append road name: sbarah_sanahi with shortest path time: 640.487141575

will append road name: mnar_anes with shortest path time: 421.6175998723538
shortest_path_rehab_road_tracker is: ['mnar_anes']
will append road name: mnar_anes with shortest path time: 1228.7702325692499
shortest_path_rehab_road_tracker is: ['mnar_anes']
will append road name: mnar_anes with shortest path time: 757.3397500034271
shortest_path_rehab_road_tracker is: ['talook']
will append road name: talook with shortest path time: 826.5207076069779
shortest_path_rehab_road_tracker is: ['talook']
will append road name: talook with shortest path time: 257.88695705687724
shortest_path_rehab_road_tracker is: ['talook']
will append road name: talook with shortest path time: 347.30672292689786
shortest_path_rehab_road_tracker is: ['akmat_hubaish']
will append road name: akmat_hubaish with shortest path time: 153.51995644453288
shortest_path_rehab_road_tracker is: ['akmat_hubaish']
will append road name: akmat_hubaish with shortest path time: 168.97201706808693
shortest_path_rehab_road_tracker

shortest_path_rehab_road_tracker is: ['al_muaras_al_bajela']
will append road name: al_muaras_al_bajela with shortest path time: 229.82712166577733
shortest_path_rehab_road_tracker is: ['taiz_to_al_turbah']
will append road name: taiz_to_al_turbah with shortest path time: 95.60438950751634
shortest_path_rehab_road_tracker is: ['sbarah_sanahi']
will append road name: sbarah_sanahi with shortest path time: 734.780916171051
shortest_path_rehab_road_tracker is: ['sbarah_sanahi']
will append road name: sbarah_sanahi with shortest path time: 125.96589563761388
shortest_path_rehab_road_tracker is: ['sbarah_sanahi']
will append road name: sbarah_sanahi with shortest path time: 21.872705458993316
shortest_path_rehab_road_tracker is: ['sbarah_sanahi']
will append road name: sbarah_sanahi with shortest path time: 221.4023760606536
shortest_path_rehab_road_tracker is: ['sbarah_sanahi']
will append road name: sbarah_sanahi with shortest path time: 263.4389667254441
shortest_path_rehab_road_tracker 

will append road name: mansuriah_dir_maalm with shortest path time: 2057.5543300480713
shortest_path_rehab_road_tracker is: ['Al Shurge- Al Hammadi & Sum Branches']
will append road name: Al Shurge- Al Hammadi & Sum Branches with shortest path time: 605.5893271119882
shortest_path_rehab_road_tracker is: ['afiosh']
will append road name: afiosh with shortest path time: 1222.0666603583297
shortest_path_rehab_road_tracker is: ['modhfar_alll']
will append road name: modhfar_alll with shortest path time: 419.4135660394382
shortest_path_rehab_road_tracker is: ['modhfar_alll']
will append road name: modhfar_alll with shortest path time: 307.13194598555066
shortest_path_rehab_road_tracker is: ['modhfar_alll']
will append road name: modhfar_alll with shortest path time: 141.31606113036509
shortest_path_rehab_road_tracker is: ['modhfar_alll']
will append road name: modhfar_alll with shortest path time: 64.68106512275935
shortest_path_rehab_road_tracker is: ['modhfar_alll']
will append road name:

shortest_path_rehab_road_tracker is: ['hajdah_rasian']
will append road name: hajdah_rasian with shortest path time: 368.16132033914533
shortest_path_rehab_road_tracker is: ['hajdah_rasian']
will append road name: hajdah_rasian with shortest path time: 385.75908366343543
shortest_path_rehab_road_tracker is: ['hajdah_rasian']
will append road name: hajdah_rasian with shortest path time: 2054.713521786832
shortest_path_rehab_road_tracker is: ['hajdah_rasian']
will append road name: hajdah_rasian with shortest path time: 145.5289693162933
shortest_path_rehab_road_tracker is: ['jbl_murad']
will append road name: jbl_murad with shortest path time: 368.7380322032343
shortest_path_rehab_road_tracker is: ['hajdah_rasian']
will append road name: hajdah_rasian with shortest path time: 322.01038269745635
shortest_path_rehab_road_tracker is: ['taiz_to_al_turbah']
will append road name: taiz_to_al_turbah with shortest path time: 1011.1073643484885
shortest_path_rehab_road_tracker is: ['taiz_to_al_t

will append road name: ribat_hiwah_mnwz with shortest path time: 448.3404720570041
shortest_path_rehab_road_tracker is: ['ribat_hiwah_mnwz']
will append road name: ribat_hiwah_mnwz with shortest path time: 403.2021356980922
shortest_path_rehab_road_tracker is: ['ribat_hiwah_mnwz']
will append road name: ribat_hiwah_mnwz with shortest path time: 607.3867169279536
shortest_path_rehab_road_tracker is: ['ribat_hiwah_mnwz']
will append road name: ribat_hiwah_mnwz with shortest path time: 520.0532730087441
shortest_path_rehab_road_tracker is: ['ribat_hiwah_mnwz']
will append road name: ribat_hiwah_mnwz with shortest path time: 759.383485228635
shortest_path_rehab_road_tracker is: ['ribat_hiwah_mnwz']
will append road name: ribat_hiwah_mnwz with shortest path time: 637.1078233079265
shortest_path_rehab_road_tracker is: ['ribat_hiwah_mnwz']
will append road name: ribat_hiwah_mnwz with shortest path time: 417.03865575191503
shortest_path_rehab_road_tracker is: ['ribat_hiwah_mnwz']
will append r

will append road name: ribat_hiwah_mnwz with shortest path time: 1327.8640130630447
shortest_path_rehab_road_tracker is: ['ribat_hiwah_mnwz']
will append road name: ribat_hiwah_mnwz with shortest path time: 535.7967166846662
shortest_path_rehab_road_tracker is: ['ribat_hiwah_mnwz']
will append road name: ribat_hiwah_mnwz with shortest path time: 1966.8242739417333
shortest_path_rehab_road_tracker is: ['Al Shurge- Al Hammadi & Sum Branches']
will append road name: Al Shurge- Al Hammadi & Sum Branches with shortest path time: 47.21279578148011
shortest_path_rehab_road_tracker is: ['hajdah_rasian']
will append road name: hajdah_rasian with shortest path time: 66.38137277156643
shortest_path_rehab_road_tracker is: ['hajdah_rasian']
will append road name: hajdah_rasian with shortest path time: 399.2566172112986
shortest_path_rehab_road_tracker is: ['hajdah_rasian']
will append road name: hajdah_rasian with shortest path time: 325.12076775658176
shortest_path_rehab_road_tracker is: ['hajdah_

will append road name: serhan with shortest path time: 520.5357118557951
shortest_path_rehab_road_tracker is: ['serhan']
will append road name: serhan with shortest path time: 193.2178604383643
shortest_path_rehab_road_tracker is: ['serhan']
will append road name: serhan with shortest path time: 4167.910933776305
shortest_path_rehab_road_tracker is: ['serhan']
will append road name: serhan with shortest path time: 623.7453158998134
shortest_path_rehab_road_tracker is: ['misrakh']
will append road name: misrakh with shortest path time: 113.5200647066595
shortest_path_rehab_road_tracker is: ['afiosh']
will append road name: afiosh with shortest path time: 607.5632964456806
shortest_path_rehab_road_tracker is: ['afiosh']
will append road name: afiosh with shortest path time: 2482.4187128025796
shortest_path_rehab_road_tracker is: ['afiosh']
will append road name: afiosh with shortest path time: 680.124424494285
shortest_path_rehab_road_tracker is: ['afiosh']
will append road name: afiosh 

shortest_path_rehab_road_tracker is: ['ribat_hiwah_mnwz']
will append road name: ribat_hiwah_mnwz with shortest path time: 1034.5009534753315
shortest_path_rehab_road_tracker is: ['ribat_hiwah_mnwz']
will append road name: ribat_hiwah_mnwz with shortest path time: 151.89799714434469
shortest_path_rehab_road_tracker is: ['ribat_hiwah_mnwz']
will append road name: ribat_hiwah_mnwz with shortest path time: 479.52010634229987
shortest_path_rehab_road_tracker is: ['ribat_hiwah_mnwz']
will append road name: ribat_hiwah_mnwz with shortest path time: 599.2712472714561
shortest_path_rehab_road_tracker is: ['ribat_hiwah_mnwz']
will append road name: ribat_hiwah_mnwz with shortest path time: 2034.3339426375442
shortest_path_rehab_road_tracker is: ['ribat_hiwah_mnwz']
will append road name: ribat_hiwah_mnwz with shortest path time: 864.4134569870291
shortest_path_rehab_road_tracker is: ['ourod']
will append road name: ourod with shortest path time: 545.693993002618
shortest_path_rehab_road_tracker

will append road name: taiz_to_al_turbah with shortest path time: 222.2865997157051
shortest_path_rehab_road_tracker is: ['Al Saheed Mountain- Bani Falah- Bani Zaid Alhada']
will append road name: Al Saheed Mountain- Bani Falah- Bani Zaid Alhada with shortest path time: 1682.3203995418953
shortest_path_rehab_road_tracker is: ['shriki_hamadi']
will append road name: shriki_hamadi with shortest path time: 1554.5988255121445
shortest_path_rehab_road_tracker is: ['taiz_to_al_turbah']
will append road name: taiz_to_al_turbah with shortest path time: 756.3567153752435
shortest_path_rehab_road_tracker is: ['taiz_to_al_turbah']
will append road name: taiz_to_al_turbah with shortest path time: 1302.0694646983854
shortest_path_rehab_road_tracker is: ['afiosh']
will append road name: afiosh with shortest path time: 3189.8547265761845
shortest_path_rehab_road_tracker is: ['mansuriah_dir_maalm']
will append road name: mansuriah_dir_maalm with shortest path time: 1087.4146451086929
shortest_path_reh

In [151]:
mismatch_count

0

In [152]:
rehab_road_tracker

defaultdict(list,
            {'hajdah_rasian': [1785.915527673768,
              825.4324168391543,
              2062.3935276004354,
              473.85954281937546,
              1107.8349638926718,
              1243.250952204997,
              622.7868215277356,
              2054.9254854978344,
              467.4055337065263,
              266.1475355268018,
              67.19809241499739,
              146.65581784292542,
              339.5009900381386,
              223.83017872019778,
              813.6744171734085,
              523.8477827890273,
              268.4771440937854,
              454.07667999680245,
              241.04512752357442,
              221.86289404763835,
              212.73375498072323,
              353.48242238470584,
              95.64044742496272,
              184.71084153405698,
              481.6357764013224,
              1035.817903324261,
              1880.3915000102481,
              966.9266178339158,
              368.1613203391

In [153]:
rehab_road_tracker

defaultdict(list,
            {'hajdah_rasian': [1785.915527673768,
              825.4324168391543,
              2062.3935276004354,
              473.85954281937546,
              1107.8349638926718,
              1243.250952204997,
              622.7868215277356,
              2054.9254854978344,
              467.4055337065263,
              266.1475355268018,
              67.19809241499739,
              146.65581784292542,
              339.5009900381386,
              223.83017872019778,
              813.6744171734085,
              523.8477827890273,
              268.4771440937854,
              454.07667999680245,
              241.04512752357442,
              221.86289404763835,
              212.73375498072323,
              353.48242238470584,
              95.64044742496272,
              184.71084153405698,
              481.6357764013224,
              1035.817903324261,
              1880.3915000102481,
              966.9266178339158,
              368.1613203391

In [154]:
rehab_road_tracker_mean = {}

In [155]:
from statistics import mean

In [156]:
rehab_road_tracker_export = {}
for key, value in rehab_road_tracker.items():
    print(key)
    print(mean(value))
    print(len(value))
    rehab_road_tracker_export[key]=[mean(value),len(value)]

hajdah_rasian
665.5052107592076
204
mnar_anes
804.8795277135325
116
sbarah_sanahi
704.6650541546255
76
bajil_assaleef
1752.651324745966
32
Al Shurge- Al Hammadi & Sum Branches
719.5660525713473
136
afiosh
1102.2159879355802
273
ribat_hiwah_mnwz
1049.4774341459427
174
akmat_hubaish
356.84121453774793
41
shriki_hamadi
1479.8475627002515
160
taiz_to_al_turbah
819.4617105805638
427
mansuriah_dir_maalm
1193.0928188591108
79
alaman_suqe_alamasha
316.0835623500796
30
al_muaras_al_bajela
670.1345478893995
11
jabal_arrabowah
633.9823730651508
9
jbl_murad
749.1020403713864
46
al_zone_jarmah_track
1277.8545632128994
21
morais
991.581766963965
1
talook
335.8979644984895
70
ourod
541.0139800205624
108
modhfar_alll
1111.2359969999254
21
misrakh
510.8136549536889
59
serhan
1001.640202899914
25
kir_all
462.27329911957827
9
2turbah_tor_albaha
729.77672966662
74
kaniah_al_abdiah
1140.0365477977543
37
ba_tays_rousod
1161.2595792195957
13
AL-Baida Mukel
1430.7620554998314
5
al_abr_to_seiyun
1620.958760269

In [157]:
rehab_road_tracker_export

{'hajdah_rasian': [665.5052107592076, 204],
 'mnar_anes': [804.8795277135325, 116],
 'sbarah_sanahi': [704.6650541546255, 76],
 'bajil_assaleef': [1752.651324745966, 32],
 'Al Shurge- Al Hammadi & Sum Branches': [719.5660525713473, 136],
 'afiosh': [1102.2159879355802, 273],
 'ribat_hiwah_mnwz': [1049.4774341459427, 174],
 'akmat_hubaish': [356.84121453774793, 41],
 'shriki_hamadi': [1479.8475627002515, 160],
 'taiz_to_al_turbah': [819.4617105805638, 427],
 'mansuriah_dir_maalm': [1193.0928188591108, 79],
 'alaman_suqe_alamasha': [316.0835623500796, 30],
 'al_muaras_al_bajela': [670.1345478893995, 11],
 'jabal_arrabowah': [633.9823730651508, 9],
 'jbl_murad': [749.1020403713864, 46],
 'al_zone_jarmah_track': [1277.8545632128994, 21],
 'morais': [991.581766963965, 1],
 'talook': [335.8979644984895, 70],
 'ourod': [541.0139800205624, 108],
 'modhfar_alll': [1111.2359969999254, 21],
 'misrakh': [510.8136549536889, 59],
 'serhan': [1001.640202899914, 25],
 'kir_all': [462.27329911957827, 9

In [158]:
rehab_road_tracker_export_df = pd.DataFrame.from_dict(rehab_road_tracker_export, orient='index',
                       columns=['mean_travel_time', 'num_of_trips'])

In [159]:
rehab_road_tracker_export_df.to_csv(r'output2/rehab_road_tracker_health_38N.csv')

In [160]:
rehab_origin_destination_pairs

[(6936545287.0, 9990018857),
 (1217921704.0, 9990006478),
 (1226311587.0, 9990011008),
 (1226311656.0, 9990011008),
 (1226311682.0, 9990011008),
 (1226312289.0, 9990011008),
 (1226312302.0, 9990011008),
 (1226312343.0, 9990011008),
 (1224740642.0, 9990031342),
 (1224741436.0, 9990031342),
 (1224741480.0, 9990040896),
 (6936546113.0, 9990015563),
 (6936546165.0, 9990005858),
 (7294963921.0, 9990011008),
 (7294964000.0, 9990011008),
 (7294949599.0, 9990045023),
 (7294949616.0, '109871'),
 (7294949625.0, 9990006432),
 (7294949638.0, 9990001679),
 (7294949663.0, 'new_obj_263'),
 (7294949677.0, 9990000241),
 (7294949681.0, 9990000241),
 (7294949682.0, 9990024852),
 (7294949688.0, 9990000241),
 (7294949696.0, 9990000241),
 (7294949705.0, 9990000241),
 (7294949708.0, 9990000241),
 (7294949712.0, 9990045023),
 (7294949718.0, 9990045609),
 (7294949752.0, 9990011008),
 (7294949753.0, 9990011008),
 (7294949759.0, 9990046789),
 (7294949762.0, 9990047170),
 (7294949779.0, 9990047170),
 (7294949794.

#### Recorded all of the origin and destination IDs of the paths that pass through the rehab roads in rehab_road_tracker, now run again using these same ids but using the regular graph for a comparison

In [161]:
reg_road_tracker = defaultdict(list)

In [162]:
def look_up2(shortest_path, shortest_path_time):
    shortest_path_road_tracker = []
    edge_segements = list(group(shortest_path,2))
    for edge in edge_segements:
        try:           
            if G_adv_w_time[edge[0]][edge[1]][0]['Type'] == 'rehab':
                print('rehab match')
                road_name = G_adv_w_time[edge[0]][edge[1]][0]['name']
                shortest_path_road_tracker.append(road_name)
                #reg_road_tracker[road_name].append(shortest_path_time)
                #break
        except:
            print(G_adv_w_time[edge[0]][edge[1]][0])
            print(edge[0])
            print(edge[1])
            global mismatch_count2
            mismatch_count2 += 1
            
    if len(shortest_path_road_tracker) > 0:
        
        shortest_path_road_tracker = list(set(shortest_path_road_tracker))
        
        if len(shortest_path_road_tracker) > 1:
            print('This shortest path traversed more than one reg roads')
        
        print(f"shortest_path_road_tracker is: {shortest_path_road_tracker}")

        for reg_road in shortest_path_road_tracker:
            print(f"will append road name: {reg_road} with shortest path time: {shortest_path_time}")
            reg_road_tracker[reg_road].append(shortest_path_time)

In [163]:
%time
mismatch_count2 = 0
for origin, destination in rehab_origin_destination_pairs:
    shortest_path = nx.dijkstra_path(G_adv_w_time, origin, destination, weight='time')
    shortest_path_time = OD_df_reg.loc[origin, destination]
    look_up2(shortest_path, shortest_path_time)

CPU times: user 1e+03 ns, sys: 0 ns, total: 1e+03 ns
Wall time: 3.81 µs
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
shortest_path_road_tracker is: ['hajdah_rasian']
will append road name: hajdah_rasian with shortest path time: 2056.748870234117
rehab match
rehab match
rehab match
rehab match
rehab match
shortest_path_road_tracker is: ['mnar_anes']
will append road name: mnar_anes with shortest path time: 3581.293897030908
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
shortest_path_road_tracker is: ['sbarah_sanahi']
will append road name: sbarah_sanahi with shortest path time: 404.7899961713875
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
shortest_path_road_tracker is: ['sbarah_sanahi']
will append road name: sbarah_sanahi with shortest path time: 763.753413795556
r

shortest_path_road_tracker is: ['bajil_assaleef']
will append road name: bajil_assaleef with shortest path time: 3204.107987758559
rehab match
rehab match
rehab match
shortest_path_road_tracker is: ['bajil_assaleef']
will append road name: bajil_assaleef with shortest path time: 1814.7586594791874
rehab match
shortest_path_road_tracker is: ['bajil_assaleef']
will append road name: bajil_assaleef with shortest path time: 5300.345479501729
rehab match
shortest_path_road_tracker is: ['bajil_assaleef']
will append road name: bajil_assaleef with shortest path time: 4512.025208517191
rehab match
shortest_path_road_tracker is: ['bajil_assaleef']
will append road name: bajil_assaleef with shortest path time: 4399.712526055194
rehab match
shortest_path_road_tracker is: ['bajil_assaleef']
will append road name: bajil_assaleef with shortest path time: 2639.5188895014935
rehab match
rehab match
rehab match
shortest_path_road_tracker is: ['mnar_anes']
will append road name: mnar_anes with shortest 

rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
shortest_path_road_tracker is: ['mnar_anes']
will append road name: mnar_anes with shortest path time: 686.9808010157636
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
shortest_path_road_tracker is: ['mnar_anes']
will append road name: mnar_anes with shortest path time: 1601.1271393280963
rehab match
rehab match
rehab match
rehab match
rehab match
shortest_path_road_tracker is: ['mnar_anes']
will append road name: mnar_anes with shortest path time: 2336.317292600023
rehab match
rehab match
shortest_path_road_tracker is: ['talook']
will append road name: talook with shortest path time: 902.5780352774603
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
shortest_path_road_tracker is: ['talook']
will append road name: talook with shortest path time: 356.7941689102046
rehab match
rehab match
rehab match
rehab match
rehab match
rehab matc

rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
shortest_path_road_tracker is: ['shriki_hamadi']
will append road name: shriki_hamadi with shortest path time: 1604.7102451733804
rehab match
rehab match
shortest_path_road_tracker is: ['afiosh']
will append road name: afiosh with shortest path time: 812.9751092334386
rehab match
shortest_path_road_tracker is: ['afiosh']
will append road name: afiosh with shortest path time: 185.66861138268496
rehab match
rehab match
rehab match
shortest_path_road_tracker is: ['afiosh']
will append road name

rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
shortest_path_road_tracker is: ['shriki_hamadi']
will append road name: shriki_hamadi with shortest path time: 1223.16487334798
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
shortest_path_road_tracker is: ['sbarah_sanahi']
will append road name: sbarah_sanahi with shortest path time: 1630.9660818693028
rehab match
shortest_path_road_tracker is: ['afiosh']
will append road name: afiosh with shortest path time: 461.9176050179405
rehab match
shortest_path_road_tracker is: ['alaman_suqe_alamasha']
will append road name: alaman_suqe_alamasha with shorte

...

rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
shortest_path_road_tracker is: ['ribat_hiwah_mnwz']
will append road name: ribat_hiwah_mnwz with shortest path time: 957.8635325512774
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
shortest_path_road_tracker is: ['ribat_hiwah_mnwz']
will append road name: ribat_hiwah_mnwz with shortest path time: 2378.1252725825775
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
shortest_path_road_tracker is: ['ribat_hiwah_mnwz']
will append road name: ribat_hiwah_mnwz with shortest path time: 1208.2047869320631
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
shortest_path_road_tracker is: ['ourod']
will append road name: ourod with shortest path time: 716.6718526809717
rehab match
rehab match
rehab match
rehab match


rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
shortest_path_road_tracker is: ['shriki_hamadi']
will append road name: shriki_hamadi with shortest path time: 3559.951442035889
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
shortest_path_road_tracker is: ['shriki_hamadi']
will append road name: shriki_hamadi with shortest path time: 3140.2869707533127
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab mat

rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
shortest_path_road_tracker is: ['shriki_hamadi']
will append road name: shriki_hamadi with shortest path time: 165.73149059121837
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
shortest_path_road_tracker is: ['taiz_to_al_turbah']
will append road name: taiz_to_al_turbah with shortest path time: 1095.5979518336721
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
shortest_path_road_tracker is: ['hajdah_rasian']
will append road name: hajdah_rasian with shortest path time: 614.56648

shortest_path_road_tracker is: ['taiz_to_al_turbah']
will append road name: taiz_to_al_turbah with shortest path time: 707.992280621501
rehab match
shortest_path_road_tracker is: ['taiz_to_al_turbah']
will append road name: taiz_to_al_turbah with shortest path time: 774.2382180254627
rehab match
rehab match
shortest_path_road_tracker is: ['talook']
will append road name: talook with shortest path time: 103.50969450892455
rehab match
rehab match
rehab match
rehab match
shortest_path_road_tracker is: ['talook']
will append road name: talook with shortest path time: 662.5842090395467
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
shortest_path_road_tracker is: ['taiz_to_al_turbah']
will append road name: taiz_to_al_turbah with shortest path time: 289.1201076234839
rehab match
rehab match
rehab match
rehab match
shortest_path_road_tracker is: ['2turbah_tor_albaha']
will append road name: 2turbah_tor_albaha with shortest path time: 2296.40498

rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
rehab match
shortest_path_road_tracker is: ['akmat_hubaish']
will append road name: akmat_hubaish with shortest path time: 751.5767432421014
rehab match
shortest_path_road_tracker is: ['ba_tays_rousod']
will append road name: ba_tays_rousod with shortest path time: 127.03178637874667
rehab match
rehab match
shortest_path_road_tracker is: ['jabal_arrabowah']
will append road name: jabal_arrabowah with shortest path time: 1217.2580293677631
rehab match
shortest_path_road_tracker is: ['jabal_arrabowah']
will append road name: jabal_arrabowah with shortest path time: 743.5711772530055
rehab match
rehab match
shortest_path_road_tracker is: ['jabal_arrabowah']
will append road name: jabal_arrabowah with shortest path time: 1394.6116159480403
rehab match
rehab match
shortest_path_road_tracker is: ['akmat_hubaish']
w

In [164]:
reg_road_tracker_export = {}
for key, value in reg_road_tracker.items():
    print(key)
    print(mean(value))
    print(len(value))
    reg_road_tracker_export[key]=[mean(value),len(value)]

hajdah_rasian
910.1085227542956
204
mnar_anes
1419.8688729305447
115
sbarah_sanahi
908.5073366625937
76
bajil_assaleef
2335.636768635806
32
Al Shurge- Al Hammadi & Sum Branches
896.2587465725272
136
afiosh
825.7068289249831
204
ribat_hiwah_mnwz
1356.7153967378683
174
akmat_hubaish
726.1512283024279
41
shriki_hamadi
1732.404781412535
160
taiz_to_al_turbah
868.054508210551
397
mansuriah_dir_maalm
1467.0241409266937
79
alaman_suqe_alamasha
544.6077598216411
30
al_muaras_al_bajela
1121.4640705143127
11
jabal_arrabowah
1098.7898892927221
9
jbl_murad
890.2836675777123
46
al_zone_jarmah_track
1606.763904808035
21
morais
1054.1125458087138
1
talook
387.92617272043515
70
ourod
725.8337461381905
95
modhfar_alll
1252.2831091200605
21
misrakh
628.279192064561
59
serhan
1216.3276831373776
25
kir_all
892.1112701370288
9
2turbah_tor_albaha
919.1289946625782
74
kaniah_al_abdiah
1390.3024838560639
37
ba_tays_rousod
1281.4413043589475
13
AL-Baida Mukel
2557.218476578117
5
al_abr_to_seiyun
2414.122220652

In [165]:
reg_road_tracker_export_df = pd.DataFrame.from_dict(reg_road_tracker_export, orient='index',
                       columns=['mean_travel_time', 'num_of_trips'])

In [166]:
reg_road_tracker_export_df.to_csv(r'output2/reg_road_tracker_health_38N.csv')